# 实践任务：
![](https://ai-studio-static-online.cdn.bcebos.com/ed6691c01b5141b1940645c613c15045fb20ae2643164862b9bad94787e2cbf2)

# 环境介绍

python==3.7 

paddle==2.2.2

# 数据集介绍
训练数据集为华南理工大学实验室公布的数据集

数据中包含500张女生图片，分别由70人进行打分，最终取平均值即为该图片的打分情况。

我们在实践中将图片分值设定为1-5。

500张图片中，450张用于训练，50张用于验证。

![](https://ai-studio-static-online.cdn.bcebos.com/9d213946134f4fc4abba86a4f5c8de829cf6b62b9d074fceb6d22c2e1c8fb71e)

**任务：分析、利用给定的数据集，训练一个人脸颜值打分模型，给出模型在验证集上的准确率，并利用模型给 work/1.jpg 图片打分**


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 配置环境

In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data
!pip install paddlex==2.0.0

data18736  dataset
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/57/f4/a69c20ee4f660081a7dedb1ac57f29be9378e04edfcb90c526b923d4bebc/beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/49/37/673d6490efc51ec46d198c75903d99de59baffdd47aea3d071b80a9e4e89/soupsieve-2.4.1-py3-none-any.whl (36 kB)

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

# 导包

In [ ]:
#导入需要的包
import os
import zipfile
import random
import json
import cv2
import numpy as np
from PIL import Image
import paddle
import matplotlib.pyplot as plt
from paddle.io import Dataset
import shutil

import paddle.fluid as fluid
from paddle.fluid.dygraph import Linear
import paddlex as pdx
import pandas as pd

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:136: UserWarning: Currently not a parallel execution environm

[06-02 15:50:06 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: FutureWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddleslim/common/analyze_helper.py:22: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://matplotlib_inline.backend_inline' by the following code:
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(cod

# 参数初始化

In [ ]:
'''
参数配置
'''
train_parameters = {
    "input_size": [3,224,224],                           #输入图片的shape
    "class_dim": -1,                                     #分类数
    "src_path":"data/data18736/face_data_5.zip",       #原始数据集路径
    "target_path":"/home/aistudio/data/dataset",        #要解压的路径 
    "train_list_path": "./train.txt",              #train_data.txt路径
    "eval_list_path": "./eval.txt",                  #eval_data.txt路径
    "label_list_path": "./label.txt",                  #eval_data.txt路径
    "test_list_path": "./test.txt",                  #test_data.txt路径
    "label_dict":{},                                    #标签字典
    "readme_path": "/home/aistudio/data/readme.json",   #readme.json路径
    "num_epochs": 40,                                    #训练轮数
    "train_batch_size": 16,                             #批次的大小
    "learning_strategy": {                              #优化函数相关的配置
        "lr": 0.01                                     #超参数学习率
    } ,
    "checkpoints": "/home/aistudio/work/checkpoints",          #保存的路径
    "skip_steps": 10,                                     #训练时输出日志的间隔
    "save_steps": 100,                                        #训练时保存模型参数的间隔
    "class_path":  "/home/aistudio/data_new",   #按分数分类的新的图片地址
    "augment_path":"/home/aistudio/data/dataset/face_data_5/face_image_train"   #数据增强图片目录
}

# 定义解压函数unzip_data

In [ ]:
def unzip_data(src_path,target_path):

    '''
    解压原始数据集，将src_path路径下的zip包解压至data目录下
    '''
    if(not os.path.isdir(target_path)):    
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()
    else:
        print("文件已解压")

# 定义划分基础数据列表函数get_data_list1

In [ ]:
#划分基础数据列表
from shutil import copyfileobj
def get_data_list1(target_path,train_list_path,val_list_path,label_list_path):
    '''
    生成数据列表
    '''
    #存储要写进eval.txt和train.txt中的内容
    trainer_list=[]
    eval_list=[]
    label_list=[]

    #获取所有类别保存的文件夹名称
    data_list_path=target_path
    class_dirs = os.listdir(data_list_path)
    
    #print(target_path)
    if '__MACOSX' in class_dirs:
        class_dirs.remove('__MACOSX')


    #读取每个类别
    for class_dir in class_dirs:
        if class_dir != ".DS_Store":
            eval_sum = 0
            trainer_sum = 0

            #统计每个类别有多少张图片
            class_sum = 0

            #获取类别路径 
            path = os.path.join(data_list_path,class_dir)
            #print(path)
            # 获取所有图片
            img_paths = os.listdir(path)
            for img_path in img_paths:                                  # 遍历文件夹下的每个图片
                if (img_path =='.DS_Store')|(img_path=='face_image_test'):
                    continue
                name_path = os.path.join(path,img_path)                       # 每张图片的路径
                print(name_path)
                for x in os.listdir(name_path):

                    y=os.path.join(name_path,x)
                    print(y)
                    if class_sum % 10 == 0:                                 # 每10张图片取一个做验证数据
                        eval_sum += 1                                       # eval_sum为测试数据的数目
                        new_eval='/home/aistudio/eval/'+x 
                        print(new_eval)
                        shutil.copy(y,new_eval)  
                        eval_list.append(new_eval + "\t%d" % int(x[0],10) + "\n")
                    else:
                        trainer_sum += 1
                        new_train='/home/aistudio/train/'+x 
                        shutil.copy(y,new_train)   
                        trainer_list.append(new_train + "\t%d" % int(x[0],10) + "\n")#trainer_sum测试数据的数目
                    class_sum += 1                                          #每类图片的数目
            
    for i in range(1,6):
        label_list.append(str(i)+"\n")

    #print(train_parameters)
    #乱序  
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image) 
    #乱序        
    random.shuffle(trainer_list) 
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image)

    with open(label_list_path, 'a') as f2:
        for label in label_list:
            f2.write(label)  


    print ('生成数据列表完成！')


# 解压

In [ ]:
# 解压

In [ ]:

'''
参数初始化
'''
src_path=train_parameters['src_path']
target_path=train_parameters['target_path']
train_list_path=train_parameters['train_list_path']
eval_list_path=train_parameters['eval_list_path']
label_list_path=train_parameters['label_list_path']
batch_size=train_parameters['train_batch_size']
class_path=train_parameters['class_path']

'''
解压原始数据到指定路径
'''
train_parameters["target_path"]="/home/aistudio/data/dataset"
target_path=train_parameters["target_path"]
unzip_data(src_path,target_path)



文件已解压


# 获取基础数据列表

In [ ]:
#每次生成数据列表前，首先清空train.txt和eval.txt

with open(train_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(eval_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 
with open(label_list_path, 'w') as f: 
    f.seek(0)
    f.truncate() 


if not os.path.exists('/home/aistudio/train'): 
    os.mkdir(r'/home/aistudio/train')
    os.mkdir(r'/home/aistudio/eval')
    train_parameters["target_path"]="/home/aistudio/data/dataset"
    target_path=train_parameters["target_path"]
    print("生成数据列表 ")
    #生成数据列表  
    get_data_list1(target_path,train_list_path,eval_list_path,label_list_path)



生成数据列表 
/home/aistudio/data/dataset/face_data_5/face_image_train
/home/aistudio/data/dataset/face_data_5/face_image_train/3-68.jpg
/home/aistudio/eval/3-68.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-231.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-14.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-198.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/3-66.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-388.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/3-20.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/3-378.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-157.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-455.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-423.jpg
/home/aistudio/eval/2-423.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/2-175.jpg
/home/aistudio/data/dataset/face_data_5/face_image_train/3-308.jpg
/home/ais

In [ ]:
def proc_img(src):
    for root, dirs, files in os.walk(src):
        if '__MACOSX' in root:continue
        for file in files:            
            src=os.path.join(root,file)
            img=Image.open(src)
            if img.mode != 'RGB': 
                    img = img.convert('RGB') 
                    img.save(src)            

if __name__=='__main__':
    proc_img(r"/home/aistudio/data_new")

# 做数据增强

In [ ]:
!pip install Augmentor

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import Augmentor,glob,os,shutil
augment_path=train_parameters['augment_path']
# 设置输出目录，控制不重复增强数据


path_ ="/home/aistudio/train"

print("path_",path_)
print('数据增强：',path_)
print('image：',path_)

#创建 Augmentor.Pipeline 实例，并指定需要增强的图片所在目录，设置增强操作的参数，如旋转、缩放、扭曲等
p = Augmentor.Pipeline(path_,output_directory='output')
b = Augmentor.Pipeline(path_,output_directory='output')
c = Augmentor.Pipeline(path_,output_directory='output')
#ouput为中间目录，最后需要删除


#p.rotate()方法来设置旋转增强的概率（probability）
#最大左旋角度（max_left_rotation）
#最大右旋角度（max_right_rotation）
p.rotate(probability=0.6, max_left_rotation=2, max_right_rotation=2)

#p.zoom()方法用于设置缩放增强的概率（probability）
#最小缩放因子（min_factor）
#最大缩放因子（max_factor）
b.zoom(probability=0.6, min_factor=0.9, max_factor=1.1)

#p.random_distortion()方法用于设置扭曲增强的概率（probability）
#网格高度（grid_height）、网格宽度（grid_width）和扭曲强度（magnitude）
c.random_distortion(probability=0.4, grid_height=2, grid_width=2, magnitude=1)

# 根据已有图片数量计算需要增强的数量
count = 200 - len(glob.glob(pathname=path_+'/*.jpg'))

#调用 sample() 方法进行样本扩增。
p.sample(count, multi_threaded=False)
p.process()
#调用 sample() 方法进行样本扩增。
b.sample(count, multi_threaded=False)
b.process()
#调用 sample() 方法进行样本扩增。
c.sample(count, multi_threaded=False)
c.process()

print('将生成的图片拷贝到正确的目录')
for root, dirs, files in os.walk("/home/aistudio/train", topdown=False):
    for name in files:
        path_ = os.path.join(root, name)
        #print(path_)
        name2=name
        substring = "train_original_"
        name2 = name.replace(substring, '')
        #print(name2) 

        if path_.rsplit('/',3)[2] == 'output':
            dest_dir = os.path.join(augment_path ,'train') 
            #print(dest_dir)
            if not os.path.exists(dest_dir):os.makedirs(dest_dir) 
            dest_path_ = os.path.join(augment_path ,name2) 
            shutil.move(path_, dest_path_)
            print(dest_path_)
print('删除所有output目录')
for root, dirs, files in os.walk("/home/aistudio/train", topdown=False):
    for name in dirs:
        if name == 'output':
            path_ = os.path.join(root, name)
            shutil.rmtree(path_)
print('完成数据增强')


path_ /home/aistudio/train
数据增强： /home/aistudio/train
image： /home/aistudio/train
Initialised with 405 image(s) found.
Output directory set to /home/aistudio/train/output.Initialised with 405 image(s) found.
Output directory set to /home/aistudio/train/output.Initialised with 405 image(s) found.
Output directory set to /home/aistudio/train/output.

Executing Pipeline: 0 Samples [00:00, ? Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F69ACE7C650>: 100%|██████████| 405/405 [00:01<00:00, 316.02 Samples/s]                 
Executing Pipeline: 0 Samples [00:00, ? Samples/s]
Processing <PIL.Image.Image image mode=RGB size=224x224 at 0x7F69AC499750>: 100%|██████████| 405/405 [00:00<00:00, 440.52 Samples/s]                 
Executing Pipeline: 0 Samples [00:00, ? Samples/s]
Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7F69AC643A50>: 100%|██████████| 405/405 [00:01<00:00, 320.25 Samples/s]


# 定义获取增强数据列表函数add_data_list

In [ ]:
#增强数据列表

def add_data_list(train_list_path,augment_path):
    '''
    生成数据列表
    '''
    #存储要写进eval.txt和train.txt中的内容
    add_list=[]

    #获取所有类别保存的文件夹名称
    add_list_path=augment_path
    add_imgs = os.listdir(add_list_path)
    
    #print(target_path)
    if 'train' in add_imgs:
        add_imgs.remove('train')
    #读取每个类别
    for add_img in add_imgs:
        if add_img != ".DS_Store":
            #获取类别路径 
            add_path = os.path.join(add_list_path,add_img)
            add_list.append(add_path + "\t%d" % int(add_img[0],10) + "\n")#trainer_sum测试数据的数目

    #print(train_parameters)
    #乱序  
    random.shuffle(add_list)
    with open(train_list_path, 'a') as f:
        for add_image in add_list:
            f.write(add_image) 
    print ('生成数据列表完成！')


# 将增广后的数据加入train_list

In [ ]:
#将增广的数据加入train_list
add_data_list(train_list_path,augment_path)

生成数据列表完成！


# 数据读取器定义

In [ ]:
from paddle.vision import transforms as T

class Reader(Dataset):
    def __init__(self, data_path, mode='train'):
        """
        数据读取器
        :param data_path: 数据集所在路径
        :param mode: train or eval
        """
        super().__init__()
        self.data_path = data_path
        self.img_paths = []
        self.labels = []
        if mode == 'train':
            with open(os.path.join(self.data_path, "train.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
            for img_info in self.info:
                img_path, label = img_info.strip().split('\t')
                self.img_paths.append(img_path)
                self.labels.append(int(label))

        else:
            with open(os.path.join(self.data_path, "eval.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
            for img_info in self.info:
                img_path, label = img_info.strip().split('\t')
                self.img_paths.append(img_path)
                self.labels.append(int(label))


    def __getitem__(self, index):
        """
        获取一组数据
        :param index: 文件索引号
        :return:
        """
        # 第一步打开图像文件并获取label值
        img_path = self.img_paths[index]
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB') 
        img = img.resize((224, 224), Image.BILINEAR)
        img = np.array(img).astype('float32')
        img = img.transpose((2, 0, 1)) / 255
        label = self.labels[index]
        label = np.array([label], dtype="int64")
        return img, label

    def print_sample(self, index: int = 0):
        print("文件名", self.img_paths[index], "\t标签值", self.labels[index])

    def __len__(self):
        return len(self.img_paths)

In [ ]:
train_dataset = Reader('/home/aistudio/',mode='train')
print(train_dataset.__len__())
eval_dataset = Reader('/home/aistudio/',mode='eval')
print(eval_dataset.__len__())
#训练数据加载
train_loader = paddle.io.DataLoader(train_dataset, batch_size=16, shuffle=True)
#测试数据加载
eval_loader = paddle.io.DataLoader(eval_dataset, batch_size = 8, shuffle=False)

2070
45


# 查看数据

In [ ]:

print(train_dataset.__len__())
print(eval_dataset.__len__())
print(eval_dataset.__getitem__(10)[0].shape)
print(eval_dataset.__getitem__(10)[1].shape)

2070
45
(3, 224, 224)
(1,)


# 使用resnet101_vd_ssld训练模型

In [ ]:
from paddlex import transforms as T
train_transforms = T.Compose(
    [T.RandomCrop(crop_size=224), T.RandomHorizontalFlip(), T.Normalize()])

eval_transforms = T.Compose([
    T.ResizeByShort(short_size=256), T.CenterCrop(crop_size=224), T.Normalize()
])

In [23]:

train_dataset = pdx.datasets.ImageNet(
    data_dir='/home/aistudio/data/dataset/face_data_5/face_image_train',
    file_list='/home/aistudio/train.txt',
    label_list='/home/aistudio/label.txt',
    transforms=train_transforms,
    shuffle=True)
    
eval_dataset = pdx.datasets.ImageNet(
    data_dir='/home/aistudio/data/dataset/face_data_5/face_image_train',
    file_list='/home/aistudio/eval.txt',
    label_list='/home/aistudio/label.txt',
    transforms=eval_transforms)
print(len(train_dataset.labels))
num_classes = 6
model = pdx.cls.ResNet101_vd_ssld(num_classes=num_classes)
model.train(num_epochs=40,
            train_dataset=train_dataset,
            train_batch_size=16,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[6, 8],
            save_interval_epochs=1,
            learning_rate=0.005,
            save_dir='output/resNet101_vd_ssld',
            use_vdl=True)

2023-06-02 16:11:44 [INFO]	Starting to read file list from dataset...
2023-06-02 16:11:44 [INFO]	2070 samples in file /home/aistudio/train.txt
2023-06-02 16:11:44 [INFO]	Starting to read file list from dataset...
2023-06-02 16:11:44 [INFO]	45 samples in file /home/aistudio/eval.txt
5
2023-06-02 16:11:44 [INFO]	Loading pretrained model from output/resNet101_vd_ssld/pretrain/ResNet101_vd_ssld_pretrained.pdparams
2023-06-02 16:11:45 [WARNING]	[SKIP] Shape of pretrained params out.weight doesn't match.(Pretrained: [2048, 1000], Actual: [2048, 6])
2023-06-02 16:11:45 [WARNING]	[SKIP] Shape of pretrained params out.bias doesn't match.(Pretrained: [1000], Actual: [6])
2023-06-02 16:11:45 [INFO]	There are 530/532 variables loaded into ResNet101_vd_ssld.
2023-06-02 16:11:47 [INFO]	[TRAIN] Epoch=1/40, Step=10/129, loss=1.060453, acc1=0.500000, acc5=1.000000, lr=0.005000, time_each_step=0.15s, eta=0:12:58
2023-06-02 16:11:48 [INFO]	[TRAIN] Epoch=1/40, Step=20/129, loss=1.221204, acc1=0.562500, ac

KeyboardInterrupt: 

In [24]:
model = pdx.load_model('output/resNet101_vd_ssld/best_model')
results = []
root_path = '/home/aistudio/data/dataset/face_data_5/face_image_test'
for idx in os.listdir(root_path):
    img_path = os.path.join(root_path,idx)
    orignal = model.predict(img_path)
    print(orignal)
    result = orignal[0]['category']
    results.append(result)

df = pd.DataFrame(results, columns=['Prediction'])
df.to_csv('/home/aistudio/result_bird.csv', index=True, index_label='QID')

2023-06-02 16:18:44 [INFO]	Model[ResNet101_vd_ssld] loaded.
[{'category_id': 2, 'category': '3', 'score': 0.96996146}]
[{'category_id': 3, 'category': '4', 'score': 0.67866117}]
[{'category_id': 3, 'category': '4', 'score': 0.99237895}]
[{'category_id': 2, 'category': '3', 'score': 0.99883837}]
[{'category_id': 4, 'category': '5', 'score': 0.544641}]
[{'category_id': 3, 'category': '4', 'score': 0.90058976}]
[{'category_id': 2, 'category': '3', 'score': 0.592996}]
[{'category_id': 2, 'category': '3', 'score': 0.96648484}]
[{'category_id': 2, 'category': '3', 'score': 0.9766209}]
[{'category_id': 4, 'category': '5', 'score': 0.86921006}]
[{'category_id': 3, 'category': '4', 'score': 0.97990555}]
[{'category_id': 2, 'category': '3', 'score': 0.97254163}]
[{'category_id': 2, 'category': '3', 'score': 0.8372049}]
[{'category_id': 4, 'category': '5', 'score': 0.5568044}]
[{'category_id': 3, 'category': '4', 'score': 0.992937}]
[{'category_id': 2, 'category': '3', 'score': 0.6102291}]
[{'cat

Exception in thread Thread-201:
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 583, in _get_data
    data = self._data_queue.get(timeout=self._timeout)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/queues.py", line 105, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py", line 505, in _thread_loop
    batch = self._get_data()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7

[{'category_id': 2, 'category': '3', 'score': 0.9923987}]
[{'category_id': 3, 'category': '4', 'score': 0.8799365}]
[{'category_id': 3, 'category': '4', 'score': 0.8282141}]
[{'category_id': 3, 'category': '4', 'score': 0.98525107}]
[{'category_id': 3, 'category': '4', 'score': 0.8384328}]
[{'category_id': 3, 'category': '4', 'score': 0.6641719}]
[{'category_id': 2, 'category': '3', 'score': 0.9989077}]
[{'category_id': 3, 'category': '4', 'score': 0.9478587}]
[{'category_id': 2, 'category': '3', 'score': 0.934828}]
[{'category_id': 3, 'category': '4', 'score': 0.9927108}]
[{'category_id': 4, 'category': '5', 'score': 0.9335557}]
[{'category_id': 3, 'category': '4', 'score': 0.9801444}]
[{'category_id': 4, 'category': '5', 'score': 0.9279965}]
[{'category_id': 2, 'category': '3', 'score': 0.97137105}]


In [ ]:

import shutil,os
 
try:
    shutil.rmtree("/home/aistudio/data/dataset")
except:
    print("无dataset")
try:
    os.remove("/home/aistudio/eval.txt")
except:
    print("无eval.txt")
try:
    os.remove("/home/aistudio/train.txt")
except:
    print("无dataset")
try:
    shutil.rmtree("/home/aistudio/eval")
except:
    print("无eval")
try:
    os.remove("/home/aistudio/label.txt")
except:
    print("无label.txt")
try:
    shutil.rmtree("/home/aistudio/train")
except:
    print("无train")
try:
    shutil.rmtree("/home/aistudio/train")
except:
    print("无train")